In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install numba==0.53  
!pip install scikit-learn==0.24
!pip install pycaret

In [ ]:
!pip install librosa

In [22]:
import numpy as np
import pandas as pd
import random as rn
import os

import matplotlib
import matplotlib.pyplot as plt
import sklearn
import sys
import seaborn as sns
import random as rn
import os
import scipy.stats as stats

from scipy.io import wavfile
import librosa
import IPython.display as ipd
import librosa.display
from tqdm.auto import tqdm
from pycaret.anomaly import *
from collections import Counter

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [23]:
%cd '/content/drive/MyDrive/Machine_fault/'

/content/drive/MyDrive/Machine_fault


In [24]:
# reproducibility

def all_seed(seed_num):
    np.random.seed(seed_num)
    rn.seed(seed_num)
    os.environ['PYTHONHASHSEED']=str(seed_num)
    # tf.random.set_seed(seed_num)

seed_num = 42
all_seed(seed_num)

# Make dataset

In [25]:
df_train = pd.read_csv('dataset/train.csv')
df_test = pd.read_csv('dataset/test.csv')

In [32]:
print(df_train.shape)

(1279, 4)


In [26]:
def extract_features(sample, sample_rate):
    # ZCR
    result = np.array([])
    zcr = np.mean(librosa.feature.zero_crossing_rate(y = sample).T, axis=0)
    result=np.hstack((result, zcr)) 

    # Chroma_stft
    stft = np.abs(librosa.stft(sample))
    chroma_stft = np.mean(librosa.feature.chroma_stft(S = stft, sr = sample_rate).T, axis=0)
    result = np.hstack((result, chroma_stft))

    # MelSpectogram
    mel = np.mean(librosa.feature.melspectrogram(y = sample, sr = sample_rate).T, axis=0)
    result = np.hstack((result, mel)) 

    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y = sample, sr = sample_rate).T, axis=0)
    result = np.hstack((result, mfcc)) 

    # Root Mean Square Value
    rms = np.mean(librosa.feature.rms(y = sample).T, axis=0)
    result = np.hstack((result, rms)) 

    return result

In [27]:
def get_features(path):

    sample, sample_rate = librosa.load(path)
    
    res1 = extract_features(sample, sample_rate)
    result = np.array(res1)
    
    return result

In [29]:
no_aug_ind = [i for i in range(0,3837,3)]
print(no_aug_ind)

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147, 150, 153, 156, 159, 162, 165, 168, 171, 174, 177, 180, 183, 186, 189, 192, 195, 198, 201, 204, 207, 210, 213, 216, 219, 222, 225, 228, 231, 234, 237, 240, 243, 246, 249, 252, 255, 258, 261, 264, 267, 270, 273, 276, 279, 282, 285, 288, 291, 294, 297, 300, 303, 306, 309, 312, 315, 318, 321, 324, 327, 330, 333, 336, 339, 342, 345, 348, 351, 354, 357, 360, 363, 366, 369, 372, 375, 378, 381, 384, 387, 390, 393, 396, 399, 402, 405, 408, 411, 414, 417, 420, 423, 426, 429, 432, 435, 438, 441, 444, 447, 450, 453, 456, 459, 462, 465, 468, 471, 474, 477, 480, 483, 486, 489, 492, 495, 498, 501, 504, 507, 510, 513, 516, 519, 522, 525, 528, 531, 534, 537, 540, 543, 546, 549, 552, 555, 558, 561, 564, 567, 570, 573, 576, 579, 582, 585, 588, 591, 594, 597, 600, 603, 606, 609, 612, 615, 618, 62

In [30]:
X = np.load('X.npy')
X = X[no_aug_ind]

In [31]:
print("Shape of X:", np.shape(X))

Shape of X: (1279, 162)


# **Test**

In [33]:
X_test = np.load('X_test.npy')

print("Shape of X test:", np.shape(X_test))

Shape of X test: (1514, 162)


In [34]:
train = pd.DataFrame(X, columns = [f"Col{i}" for i in range(1, 163)])
test = pd.DataFrame(X_test, columns = [f"Col{i}" for i in range(1, 163)])

In [ ]:
anom = setup(data = train, verbose = 0, silent = True, session_id = seed_num, normalize = True)

In [36]:
models()

INFO:logs:gpu_param set to False


,Name,Reference
ID,,
abod,Angle-base Outlier Detection,pyod.models.abod.ABOD
cluster,Clustering-Based Local Outlier,pyod.models.cblof.CBLOF
cof,Connectivity-Based Local Outlier,pyod.models.cof.COF
iforest,Isolation Forest,pyod.models.iforest.IForest
histogram,Histogram-based Outlier Detection,pyod.models.hbos.HBOS
knn,K-Nearest Neighbors Detector,pyod.models.knn.KNN
lof,Local Outlier Factor,pyod.models.lof.LOF
svm,One-class SVM detector,pyod.models.ocsvm.OCSVM
pca,Principal Component Analysis,pyod.models.pca.PCA


In [117]:
te_pred = np.zeros(test.shape[0], )
preds = []

for name in models()['Name'].index[:-2] :

    model = create_model(name, fraction = 0.36)

    fitting = assign_model(model)

    pred = predict_model(model, test)['Anomaly']
    preds.append(pred.tolist())
    te_pred += pred

INFO:logs:create_model_container: 62
INFO:logs:master_model_container: 62
INFO:logs:display_container: 63
INFO:logs:MCD(assume_centered=False, contamination=0.36, random_state=42,
  store_precision=True, support_fraction=None)
INFO:logs:create_model() succesfully completed......................................
INFO:logs:Initializing assign_model()
INFO:logs:assign_model(model=MCD(assume_centered=False, contamination=0.36, random_state=42,
  store_precision=True, support_fraction=None), transformation=False, score=True, verbose=True)
INFO:logs:Checking exceptions
INFO:logs:Determining Trained Model
INFO:logs:Trained Model : Minimum Covariance Determinant
INFO:logs:Copying data
INFO:logs:(1279, 164)
INFO:logs:assign_model() succesfully completed......................................
INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=MCD(assume_centered=False, contamination=0.36, random_state=42,
  store_precision=True, support_fraction=None), ml_usecase=MLUsecase.AN

In [124]:
pred = te_pred > 6

In [125]:
submission = pd.read_csv("sample_submission.csv")

In [126]:
submission['LABEL'] = pred

In [127]:
submission.to_csv('pycaret_6_0.036.csv', index = False)

In [128]:
submission.head()

,SAMPLE_ID,LABEL
0,TEST_0000,True
1,TEST_0001,False
2,TEST_0002,True
3,TEST_0003,True
4,TEST_0004,True


In [129]:
np.sum(submission['LABEL'] == 1)

922